## Pruning and Quantization
pruning and quantization could be applied on pretrained models, that is why I am thinking to use them as compression techniques other than other approaches.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()

In [4]:
net.state_dict().keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])

In [5]:
summary(net,torch.zeros([1, 3, 32, 32]))

          Kernel Shape     Output Shape   Params Mult-Adds
Layer                                                     
0_conv1   [3, 6, 5, 5]   [1, 6, 28, 28]    456.0    352.8k
1_pool               -   [1, 6, 14, 14]        -         -
2_conv2  [6, 16, 5, 5]  [1, 16, 10, 10]   2.416k    240.0k
3_pool               -    [1, 16, 5, 5]        -         -
4_fc1       [400, 120]         [1, 120]   48.12k     48.0k
5_fc2        [120, 84]          [1, 84]  10.164k    10.08k
6_fc3         [84, 10]          [1, 10]    850.0     840.0
-----------------------------------------------------------
                       Totals
Total params          62.006k
Trainable params      62.006k
Non-trainable params      0.0
Mult-Adds             651.72k


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[3, 6, 5, 5]","[1, 6, 28, 28]",456.0,352800.0
1_pool,-,"[1, 6, 14, 14]",NaN,NaN
2_conv2,"[6, 16, 5, 5]","[1, 16, 10, 10]",2416.0,240000.0
3_pool,-,"[1, 16, 5, 5]",NaN,NaN
4_fc1,"[400, 120]","[1, 120]",48120.0,48000.0
5_fc2,"[120, 84]","[1, 84]",10164.0,10080.0
6_fc3,"[84, 10]","[1, 10]",850.0,840.0


## Resnet50

In [2]:
import torchvision.models as models
model = models.resnext50_32x4d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /home/aimsp/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:11<00:00, 8.49MB/s]


In [13]:
#model.state_dict()

In [15]:
summary(model,torch.zeros([1, 3, 224, 224]))

                                             Kernel Shape       Output Shape  \
Layer                                                                          
0_conv1                                     [3, 64, 7, 7]  [1, 64, 112, 112]   
1_bn1                                                [64]  [1, 64, 112, 112]   
2_relu                                                  -  [1, 64, 112, 112]   
3_maxpool                                               -    [1, 64, 56, 56]   
4_layer1.0.Conv2d_conv1                   [64, 128, 1, 1]   [1, 128, 56, 56]   
5_layer1.0.BatchNorm2d_bn1                          [128]   [1, 128, 56, 56]   
6_layer1.0.ReLU_relu                                    -   [1, 128, 56, 56]   
7_layer1.0.Conv2d_conv2                    [4, 128, 3, 3]   [1, 128, 56, 56]   
8_layer1.0.BatchNorm2d_bn2                          [128]   [1, 128, 56, 56]   
9_layer1.0.ReLU_relu                                    -   [1, 128, 56, 56]   
10_layer1.0.Conv2d_conv3                

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[3, 64, 7, 7]","[1, 64, 112, 112]",9408.0,118013952.0
1_bn1,[64],"[1, 64, 112, 112]",128.0,64.0
2_relu,-,"[1, 64, 112, 112]",NaN,NaN
3_maxpool,-,"[1, 64, 56, 56]",NaN,NaN
4_layer1.0.Conv2d_conv1,"[64, 128, 1, 1]","[1, 128, 56, 56]",8192.0,25690112.0
5_layer1.0.BatchNorm2d_bn1,[128],"[1, 128, 56, 56]",256.0,128.0
6_layer1.0.ReLU_relu,-,"[1, 128, 56, 56]",NaN,NaN
7_layer1.0.Conv2d_conv2,"[4, 128, 3, 3]","[1, 128, 56, 56]",4608.0,14450688.0
8_layer1.0.BatchNorm2d_bn2,[128],"[1, 128, 56, 56]",256.0,128.0


In [17]:
 print(torch.__version__)

1.3.1+cpu


### Pruning

In [5]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch.nn.utils.prune'